***
# chapter 11. VIEW / SEQUENCE
***
### KEYWORD
* D
* D
* D


In [1]:
import pandas as pd
import cx_Oracle
xedb = cx_Oracle.connect('hr/hr@localhost/xe')
cur = xedb.cursor()

# [ view ]
* 오브젝트 단위의 간접 ACCESS : 실제 데이터를 구성하고 있는 테이블을 만들지 않고, 간접적으로 ACCESS

### 뷰 생성 권한 확인

In [2]:
data = pd.read_sql("SELECT * FROM session_privs",xedb)
print(data)
# CREATE VIEW 확인

               PRIVILEGE
0         CREATE SESSION
1          ALTER SESSION
2   UNLIMITED TABLESPACE
3           CREATE TABLE
4         CREATE CLUSTER
5         CREATE SYNONYM
6            CREATE VIEW
7        CREATE SEQUENCE
8   CREATE DATABASE LINK
9       CREATE PROCEDURE
10        CREATE TRIGGER
11           CREATE TYPE
12       CREATE OPERATOR
13      CREATE INDEXTYPE


### 뷰 생성 

In [3]:
cur.execute("""CREATE OR REPLACE VIEW empvw_20
                AS SELECT * FROM employees WHERE department_id=20""")


data = pd.read_sql("SELECT * FROM empvw_20",xedb)
print(data)

   EMPLOYEE_ID FIRST_NAME  LAST_NAME     EMAIL  PHONE_NUMBER  HIRE_DATE  \
0          201    Michael  Hartstein  MHARTSTE  515.123.5555 2004-02-17   
1          202        Pat        Fay      PFAY  603.123.6666 2005-08-17   

   JOB_ID  SALARY COMMISSION_PCT  MANAGER_ID  DEPARTMENT_ID  
0  MK_MAN  1000.0           None         100             20  
1  MK_REP  1000.0           None         201             20  


### 테이블인지 뷰인지 확인

In [4]:
data = pd.read_sql("""SELECT * 
                   FROM user_objects 
                   WHERE object_name IN('EMP_20','EMPVW_20')
                   """,xedb)
print(data)

  OBJECT_NAME SUBOBJECT_NAME  OBJECT_ID DATA_OBJECT_ID OBJECT_TYPE  \
0    EMPVW_20           None      20341           None        VIEW   

              CREATED       LAST_DDL_TIME            TIMESTAMP STATUS  \
0 2017-11-13 17:26:50 2017-11-13 17:26:50  2017-11-13:17:26:50  VALID   

  TEMPORARY GENERATED SECONDARY  NAMESPACE EDITION_NAME  
0         N         N         N          1         None  


In [5]:
# 뷰 상세정보 조회
data = pd.read_sql("""SELECT * 
                   FROM user_views 
                   WHERE view_name='EMPVW_20'
                   """,xedb)
print(data)

  VIEW_NAME  TEXT_LENGTH                                               TEXT  \
0  EMPVW_20          182  SELECT "EMPLOYEE_ID","FIRST_NAME","LAST_NAME",...   

  TYPE_TEXT_LENGTH TYPE_TEXT OID_TEXT_LENGTH OID_TEXT VIEW_TYPE_OWNER  \
0             None      None            None     None            None   

  VIEW_TYPE SUPERVIEW_NAME EDITIONING_VIEW READ_ONLY  
0      None           None               N         N  


### 단순뷰
* 뷰가 바라보는 테이블에 대해서 DML가능
* 함수, 표현식, 그룹함수, groupby, join 등 없음

In [6]:
cur.execute("""UPDATE empvw_20
                SET salary = 1000""")


data = pd.read_sql("SELECT salary FROM empvw_20",xedb)
print(data)

   SALARY
0  1000.0
1  1000.0


### 복합뷰
* 단순뷰 + WITH READ ONLY
* 수정불가, 조회만 가능

In [7]:
cur.execute("""CREATE OR REPLACE VIEW empvw_20
                    AS SELECT * FROM employees WHERE department_id=20
                    WITH READ ONLY""")


cur.execute("""UPDATE empvw_20
            SET salary=1000""")

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 27))



DatabaseError: ORA-42399: cannot perform a DML operation on a read-only view

### 삭제

In [12]:
cur.execute("DROP VIEW empvw_20")

# [ SEQUENCE ]
* 자동일련번호 생성

### 권한부여 확인

In [13]:
data = pd.read_sql("SELECT * FROM session_privs",xedb)
print(data)
# CREATE SEQUENCE 확인

               PRIVILEGE
0         CREATE SESSION
1          ALTER SESSION
2   UNLIMITED TABLESPACE
3           CREATE TABLE
4         CREATE CLUSTER
5         CREATE SYNONYM
6            CREATE VIEW
7        CREATE SEQUENCE
8   CREATE DATABASE LINK
9       CREATE PROCEDURE
10        CREATE TRIGGER
11           CREATE TYPE
12       CREATE OPERATOR
13      CREATE INDEXTYPE


### 시퀀스 생성

In [18]:
# 새 테이블 생성
cur.execute("""CREATE TABLE emp_seq (ID NUMBER, NAME VARCHAR2(10), DAY TIMESTAMP DEFAULT SYSTIMESTAMP)""")


# 시퀀스 생성
cur.execute("""CREATE SEQUENCE emp_id_seq
                                INCREMENT BY 1  
                                START WITH 1   
                                MAXVALUE 50  
                                NOCYCLE""") 

#증가크기1, 1부터, 50까지, 기본값 NOCYCLE(51번부터는 없다)
                
data = pd.read_sql("""SELECT * 
                   FROM user_sequences 
                   WHERE sequence_name = 'EMP_ID_SEQ'""",xedb)
print(data)

  SEQUENCE_NAME  MIN_VALUE  MAX_VALUE  INCREMENT_BY CYCLE_FLAG ORDER_FLAG  \
0    EMP_ID_SEQ          1         50             1          N          N   

   CACHE_SIZE  LAST_NUMBER  
0          20            1  


### 시퀀스 번호 순서      

In [19]:
cur.execute("""INSERT INTO emp_seq(ID, NAME, DAY) 
            VALUES(emp_id_seq.nextval, USER, DEFAULT)""")
    
data = pd.read_sql("SELECT * FROM emp_seq",xedb)
print(data) 

   ID NAME                     DAY
0   1   HR 2017-11-13 17:35:01.924


In [20]:
cur.execute("""INSERT INTO emp_seq(ID, NAME, DAY) 
            VALUES(emp_id_seq.nextval, USER, DEFAULT)""")
    
data = pd.read_sql("SELECT * FROM emp_seq",xedb)
print(data) 

   ID NAME                     DAY
0   1   HR 2017-11-13 17:35:01.924
1   2   HR 2017-11-13 17:35:04.929


In [21]:
xedb.rollback()

=> 시퀀스값 취소하지 못하기 때문에 사용한 시퀀스 번호는 재사용하지 못한다.
그 다음번호 부터 시작된다.

In [22]:
cur.execute("""INSERT INTO emp_seq(ID, NAME, DAY) 
            VALUES(emp_id_seq.nextval, USER, DEFAULT)""")

data = pd.read_sql("SELECT * FROM emp_seq",xedb)
print(data)

   ID NAME                     DAY
0   3   HR 2017-11-13 17:35:32.723


In [23]:
cur.execute("""INSERT INTO emp_seq(ID, NAME, DAY) 
            VALUES(emp_id_seq.nextval, USER, DEFAULT)""")

xedb.commit()

data = pd.read_sql("SELECT * FROM emp_seq",xedb)
print(data)

   ID NAME                     DAY
0   3   HR 2017-11-13 17:35:32.723
1   4   HR 2017-11-13 17:35:39.095


### seq.currval / seq.nextval

In [24]:
curr = pd.read_sql("SELECT emp_id_seq.currval FROM dual",xedb)
# currval : 현재사용한 번호 리턴

next = pd.read_sql("SELECT emp_id_seq.nextval FROM dual",xedb)
# nextval : 다음사용될 번호리턴

print(curr)
print(' ')
print(next)

   CURRVAL
0        4
 
   NEXTVAL
0        5


### 시퀀스 정보 수정

In [25]:
# 수정 전 MAXVALUE=50, INCREMENT BY =1
data = pd.read_sql("""SELECT * 
                   FROM user_sequences 
                   WHERE sequence_name = 'EMP_ID_SEQ'""",xedb)
print(data)

  SEQUENCE_NAME  MIN_VALUE  MAX_VALUE  INCREMENT_BY CYCLE_FLAG ORDER_FLAG  \
0    EMP_ID_SEQ          1         50             1          N          N   

   CACHE_SIZE  LAST_NUMBER  
0          20           21  


In [28]:
# MAXVALUE , INCREMENT BY 수정
cur.execute("""ALTER SEQUENCE emp_id_seq MAXVALUE 100""")
cur.execute("""ALTER SEQUENCE emp_id_seq INCREMENT BY 3""")

data = pd.read_sql("""SELECT * 
                   FROM user_sequences 
                   WHERE sequence_name = 'EMP_ID_SEQ'
                   """,xedb)
print(data)

  SEQUENCE_NAME  MIN_VALUE  MAX_VALUE  INCREMENT_BY CYCLE_FLAG ORDER_FLAG  \
0    EMP_ID_SEQ          1        100             3          N          N   

   CACHE_SIZE  LAST_NUMBER  
0          20            8  


### 삭제

In [29]:
# 테이블 삭제
cur.execute("TRUNCATE TABLE emp_seq")

# delete는 rollback을 위한 undo가 필요하지만 truncate는 DDL이라 undo가 필요하지 않다. 
# 리소스를 줄이면서 삭제하기 때문에 속도가 빠르지만 ROLLBACK 불가능하다

data = pd.read_sql("SELECT * FROM emp_seq",xedb)
print(data)

Empty DataFrame
Columns: [ID, NAME, DAY]
Index: []


In [31]:
# 시퀀스 삭제
cur.execute("DROP SEQUENCE emp_id_seq")

data = pd.read_sql("""SELECT * 
                   FROM user_sequences 
                   WHERE sequence_name = 'EMP_ID_SEQ'
                   """,xedb)
print(data)

DatabaseError: ORA-02289: sequence does not exist

In [32]:
cur.close()
xedb.close()